# Services and audit logs from Azure Services via IaC (Bicep)

A lot of the services we use in azure are fully managed and we often don't have access and audit logs for them by default. This part will cover some of the most used services and how to configure and enable logging for them.


Most Azure managed services are fairly straightforward to enable logging for, but some of them are a bit more complex. This section will cover the most common services and how to enable logging for them.
## Keyvaults, Storage Accounts, CosmosDB, Azure Openai, and similar services.

These services all follow the same pattern. They all have a "diagnostic settings" blade in the portal, where you can select which logs you want to send to which destination. The most common destinations are Log Analytics, Event Hub, and Storage Account.

To collect logs from the above services we can use the ['Microsoft.Insights/diagnosticSettings@2021-05-01-preview'](https://learn.microsoft.com/en-us/azure/templates/microsoft.insights/diagnosticsettings?pivots=deployment-language-bicep) Bicep module.

<details>
  <summary>Generic Bicep diagnostic settings template</summary>

```bicep
resource symbolicname 'Microsoft.Insights/diagnosticSettings@2021-05-01-preview' = {
  scope: resourceSymbolicName or scope
  name: 'string'
  properties: {
    eventHubAuthorizationRuleId: 'string'
    eventHubName: 'string'
    logAnalyticsDestinationType: 'string'
    logs: [
      {
        category: 'string'
        categoryGroup: 'string'
        enabled: bool
        retentionPolicy: {
          days: int
          enabled: bool
        }
      }
    ]
    marketplacePartnerId: 'string'
    metrics: [
      {
        category: 'string'
        enabled: bool
        retentionPolicy: {
          days: int
          enabled: bool
        }
        timeGrain: 'string'
      }
    ]
    serviceBusRuleId: 'string'
    storageAccountId: 'string'
    workspaceId: 'string'
  }
}
```

</details>

We can use the above template to create a diagnostic settings resource for any of the services mentioned above. I've included an example for Keyvaults here, this sends log entries to a Log Analytics workspace, The log entry contains information about what operation the user preformed, eg "GetSecret", "SetSecret". This is especially important for regulated industries, where you have strict compliance guidelines.

<details>
  <summary>Bicep for Audit Events for keyvault</summary>

```bicep
resource keyVaultdiagnosticSettings 'Microsoft.Insights/diagnosticSettings@2021-05-01-preview' = {
  scope: keyVault
  name: 'keyVaultdiagnosticSettings'
  properties: {

    workspaceId: logAnalyticsWorkspaceid
    logs: [
      {
        category: 'AuditEvent'
        enabled: true
        retentionPolicy: {
          enabled: true
          days: 120
        }
      }
    ]
  }
}
```

</details>

**Task** Now try to create a similar Bicep template for one of the other services that is used in your application stack, This could be Azure OpenAI, a storage account or a managed database. there are already templates for some resources in the repo, so you can use these are a starting point, or you can create a new one from scratch.

It should be noted that some resources create a "managed" resource group containing log analytics workspace, storage accounts or other services by default, and isn't as straight forward setting up collection of logs, as you won't have permissions on these "managed" resources. In these cases you'll need to either redeploy the resource with "self managed" services or jump though other hoops.



## App Services

App services are a bit different from the other services, as they have a built in "diagnostic settings" blade in the portal, but they also run user code. some of the below therefor also applies for container instances, k8s and vms. 

For services running any custom application that we wan't to gather logs from, we'll need to expose the application insights connection string within it. We could solve this in a few different ways.
* Hardcode the connection string in the code. This is by far the most insecure way, it means that the connection string will be in plain text in the repo.
* Add the connection string to the appsettings file. This is the easiest way, but it means that the connection string will be in plain text in the repo, it might be fine for local development, but it is not something that should be pushed to git.
* Add the connection string to the app service as an environment variable. This is a better solution, as it means that the connection string will be encrypted at rest and won't be in plain text in the repo. It also means that we can change the connection string between environments. This is a good solution for production workloads.

**Task** Instrument the app service with application insights, and add the connection string as an environment variable. This will allow us to gather logs from the app service, and it will also allow us to gather logs from the application itself.


## Protecting resources with locks

It's also a good idea to protect key vaults, audit roles and logging resources with locks. This both prevents accidental deletion and helps to ensure that the resources are not modified or deleted without proper authorization.

```bicep
resource kvDoNotDelete 'Microsoft.Authorization/locks@2017-04-01' = if (protectWithLocks) {
  scope: kv
  name: 'keyVaultDoNotDelete'
  properties: {
    level: 'CanNotDelete'
  }
}
```

#### Next steps

Let's take a look at the opentelemetry and the basics of it.
